In [6]:
import traceback

import polars as pl
import sys
import torch
import polars as pl
import numpy as np
from sklearn.linear_model import LinearRegression
from lifelines import WeibullFitter
from sklearn.model_selection import train_test_split

from torch.utils.data import Dataset, DataLoader

from utils.lstf_feature_maker.piecewise_linear_regression import PiecewiseLinearRegression
from utils.lstf_feature_maker.weibull import WeibullFeatureMaker

'''
pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128
https://developer.nvidia.com/cuda-12-8-0-download-archive
'''

MAC_DIR = '../data/'
WINDOW_DIR = 'C:/Users/USER/PycharmProjects/research/data/'

if sys.platform == 'win32':
    DIR = WINDOW_DIR
    print(torch.cuda.is_available())
    print(torch.cuda.device_count())
    print(torch.version.cuda)
    print(torch.__version__)
    print(torch.cuda.get_device_name(0))
    print(torch.__version__)
else:
    DIR = MAC_DIR

tb_bas_oper_part_mst = (pl.read_parquet(DIR + 'tb_bas_oper_part_mst.parquet')
                        .select(['OPER_PART_NO', 'OPER_PART_NM'])
                        .rename({'OPER_PART_NO': 'oper_part_no', 'OPER_PART_NM': 'oper_part_nm'}))
tb_dyn_fcst_demand = (pl.read_parquet(DIR + 'tb_dyn_fcst_dmnd.parquet')
                      .select(['PART_NO', 'DMND_QTY', 'DMND_DT', 'OPER_PART_NO'])
                      .rename({'PART_NO': 'part_no', 'OPER_PART_NO': 'oper_part_no', 'DMND_DT': 'demand_dt', 'DMND_QTY': 'demand_qty'})
                      .select(['part_no', 'oper_part_no', 'demand_dt', 'demand_qty']))
tb_dyn_fcst_demand_sellout = (pl.read_parquet(DIR + 'tb_dyn_fcst_dmnd_sellout.parquet')
                              .select(['PART_NO', 'DMND_QTY', 'DMND_DT', 'OPER_PART_NO'])
                              .rename({'PART_NO': 'part_no', 'OPER_PART_NO': 'oper_part_no', 'DMND_DT': 'demand_dt', 'DMND_QTY': 'demand_qty'})
                              .select(['part_no', 'oper_part_no', 'demand_dt', 'demand_qty']))

In [7]:
from utils.date_util import DateUtil

dyn_fcst_demand = tb_dyn_fcst_demand.with_columns([
    pl.col('demand_dt').cast(pl.Int64).map_elements(DateUtil.yyyymmdd_to_date, return_dtype = pl.Date)
])

dyn_demand_sellout = tb_dyn_fcst_demand_sellout.with_columns([
    pl.col('demand_dt').cast(pl.Int64).map_elements(DateUtil.yyyymmdd_to_date, return_dtype = pl.Date)
])

dyn_fcst_demand

part_no,oper_part_no,demand_dt,demand_qty
str,str,date,f64
"""T4240-71102BB""","""T4240-71102BB""",2018-01-01,3.0
"""T5210-34402""","""T5210-34402""",2018-01-01,1.0
"""T5210-30081""","""T5210-30081""",2018-01-01,1.0
"""T5210-65661""","""T5210-65661""",2018-01-01,1.0
"""T5210-66472""","""T5210-66472""",2018-01-01,1.0
…,…,…,…
"""U3215-52203""","""U3215-52203""",2024-02-05,30.0
"""T5710-69252""","""T5710-69252""",2024-02-05,2.0
"""DYD1-O07""","""DYD1-O07""",2024-02-05,4.0


In [8]:
dyn_fcst = (dyn_fcst_demand
                .join(tb_bas_oper_part_mst, on = 'oper_part_no', how = 'left')
                .select(['oper_part_no', 'oper_part_nm', 'demand_dt','demand_qty'])
                .sort(['oper_part_no', 'demand_dt'])
                .with_columns([
                    pl.col('demand_qty').cum_sum().over('oper_part_no').alias('cumsum_qty')
                ])
              )
dyn_demand = (dyn_demand_sellout.join(tb_bas_oper_part_mst, on = 'oper_part_no', how = 'left')
                    .select(['oper_part_no', 'oper_part_nm', 'demand_dt', 'demand_qty'])
                    .sort(['oper_part_no', 'demand_dt'])
                    .with_columns([
                        pl.col('demand_qty').cum_sum().over('oper_part_no').alias('cumsum_qty')
                 ])
               )

In [9]:
from utils.lstf_feature_maker.exponential_smoothing import ExponentialSmoothing

lookback_window = 40
horizon = 20
feature_schema = {
    'oper_part_no': pl.Utf8,
    'X_ts': pl.Array(pl.Float64, 40),
    'y_ts': pl.List(pl.Float64),
    'X_features': pl.List(pl.Float64)
}

def compute_feature_group(df: pl.DataFrame) -> pl.DataFrame:
    try:
        part_no = df['oper_part_no'][0]
        series = df['demand_qty'].to_numpy()

        # lookback_window와 horizon 확인
        if len(series) < lookback_window + horizon:
            return pl.DataFrame(schema=feature_schema)

        # Feature 계산
        LTB_point = len(series) - horizon
        slope_pre, slope_saddle, slope_post = PiecewiseLinearRegression(series = series, start_point = LTB_point).auto_piecewise_slopes()
        k, lam = WeibullFeatureMaker().auto_weibull_params(series)

        # Rolling Mean, SES
        ma9 = df['demand_qty'].rolling_mean(9).to_list()
        ses = ExponentialSmoothing().simple_exponential_smoothing(series = df['demand_qty'].to_list())

        X_ts, y_ts, X_features = [], [], []
        for i in range(len(series) - lookback_window - horizon - 1):
            X_ts.append(series[i:i+lookback_window])
            y_ts.append(series[i+lookback_window:i+lookback_window+horizon])
            X_features.append([
                slope_pre, slope_saddle, slope_post, k, lam,
                ma9[i + lookback_window], ses[i + lookback_window]
            ])

        return pl.DataFrame({
            'oper_part_no': [part_no] * len(X_ts),
            'X_ts': X_ts,
            'y_ts': y_ts,
            'X_features': X_features
        })

    except Exception as e:
        print(f"Error in compute_feature_group: {e}")
        print(traceback.print_exc())
        # return pl.DataFrame(schema=feature_schema)  # 항상 동일 스키마 반환

In [10]:
(dyn_demand
    .select(['oper_part_no', 'demand_dt', 'demand_qty', 'cumsum_qty'])
    .sort(['oper_part_no', 'demand_dt'])
    .with_columns(pl.col('demand_dt').map_elements(DateUtil.date_to_yyyymm, return_dtype = pl.Int64).alias('demand_dt'))
    .select('oper_part_no', 'demand_dt', 'demand_qty')
    .group_by(['oper_part_no', 'demand_dt'])
    .agg([
        pl.col('demand_qty').sum().alias('demand_qty')
    ])
    .sort(['oper_part_no', 'demand_dt'])
    .group_by('oper_part_no')
    .map_groups(compute_feature_group)
 )


SchemaError: type Array(Float64, 40) is incompatible with expected type List(Float64)

In [ ]:
from tqdm import tqdm
lookback = 52
horizon = 12

X_ts, y_ts, X_features = [], [], []

def compute_piecewise_slopes(values, LTB_point):
    x1 = np.arange(LTB_point).reshape(-1, 1)
    y1 = values[:LTB_point]
    slope1 = LinearRegression().fit(x1, y1).coef_[0]

    x2 = np.arange(LTB_point, len(values)).reshape(-1, 1)
    y2 = values[LTB_point:]
    slope2 = LinearRegression().fit(x2, y2).coef_[0]
    return slope1, slope2

def compute_weibull_params(values):
    failure_times = np.repeat(np.arange(1, len(values)+1), values.astype(int))
    if len(failure_times) < 5:
        return 0.0, 0.0
    wf = WeibullFitter()
    wf.fit(failure_times, event_observed=[1]*len(failure_times))
    return float(wf.rho_), float(wf.lambda_)

for part_no in df_grouped['oper_part_no'].unique():
    part_df = df_grouped.filter(pl.col("oper_part_no") == part_no).sort("yyyyww")

    series = part_df['demand_qty'].to_numpy()
    if len(series) < lookback + horizon:
        continue  # 데이터 부족 시 스킵

    # Feature 생성 (LTB 기준)
    LTB_point = len(series) - horizon
    slope_pre, slope_post = compute_piecewise_slopes(series, LTB_point)
    k, lam = compute_weibull_params(series)

    # 슬라이딩 윈도우 생성
    for i in tqdm(range(len(series) - lookback - horizon + 1)):
        X_ts.append(series[i:i+lookback])
        y_ts.append(series[i+lookback:i+lookback+horizon])
        X_features.append([slope_pre, slope_post, k, lam])

X_ts = np.array(X_ts)
y_ts = np.array(y_ts)
X_features = np.array(X_features)

print(f"X_ts shape: {X_ts.shape}, y_ts shape: {y_ts.shape}, X_features shape: {X_features.shape}")

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# scaler_ts = MinMaxScaler()
# X_ts_scaled = scaler_ts.fit_transform(X_ts.reshape(-1, 1)).reshape(X_ts.shape)
# y_ts_scaled = scaler_ts.transform(y_ts.reshape(-1, 1)).reshape(y_ts.shape)
#
# scaler_feat = StandardScaler()
# X_features_scaled = scaler_feat.fit_transform(X_features)
#
# print(X_ts_scaled)
# print(y_ts_scaled)
# print(X_features_scaled)

scaler_x = MinMaxScaler()
X_ts_scaled = scaler_x.fit_transform(X_ts.reshape(-1, 1)).reshape(X_ts.shape)

scaler_y = MinMaxScaler()
y_ts_scaled = scaler_y.fit_transform(y_ts.reshape(-1, 1)).reshape(y_ts.shape)

scaler_feat = StandardScaler()
X_features_scaled = scaler_feat.fit_transform(X_features)

In [ ]:
class DemandDataset(Dataset):
    def __init__(self, X_ts, X_feat, y_ts):
        self.X_ts = torch.tensor(X_ts, dtype = torch.float32).unsqueeze(-1)
        self.X_feat = torch.tensor(X_feat, dtype = torch.float32)
        self.y_ts = torch.tensor(y_ts, dtype = torch.float32)

    def __len__(self):
        return len(self.X_ts)

    def __getitem__(self, idx):
        return self.X_ts[idx], self.X_feat[idx], self.y_ts[idx]

X_train, X_val, F_train, F_val, y_train, y_val = train_test_split(X_ts_scaled, X_features_scaled, y_ts_scaled, test_size=0.2, random_state=42)


train_dataset = DemandDataset(X_train, F_train, y_train)
val_dataset = DemandDataset(X_val, F_val, y_val)

train_loader = DataLoader(train_dataset, batch_size = 32, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size = 32, shuffle = False)

In [ ]:
import torch
import torch.nn as nn

from layers.RevIN import RevIN


# 기존 PatchMixerLayer 그대로 사용
class PatchMixerLayer(nn.Module):
    def __init__(self, dim, a, kernel_size=8):
        super().__init__()
        self.Resnet = nn.Sequential(
            nn.Conv1d(dim, dim, kernel_size=kernel_size, groups=dim, padding='same'),
            nn.GELU(),
            nn.BatchNorm1d(dim)
        )
        self.Conv_1x1 = nn.Sequential(
            nn.Conv1d(dim, a, kernel_size=1),
            nn.GELU(),
            nn.BatchNorm1d(a)
        )

    def forward(self, x):
        x += self.Resnet(x)
        x = self.Conv_1x1(x)
        return x


# PatchMixer Backbone
class PatchMixerBackbone(nn.Module):
    def __init__(self, configs, revin=True, affine=True, subtract_last=False):
        super().__init__()
        self.n_vals = configs.enc_in
        self.lookback = configs.seq_len
        self.forecasting = configs.pred_len
        self.patch_size = configs.patch_len
        self.stride = configs.stride
        self.kernel_size = configs.mixer_kernel_size

        self.PatchMixer_blocks = nn.ModuleList([])
        self.padding_patch_layer = nn.ReplicationPad1d((0, self.stride))
        self.patch_num = int((self.lookback - self.patch_size) / self.stride + 1) + 1
        self.a = self.patch_num
        self.d_model = configs.d_model
        self.dropout_rate = configs.head_dropout
        self.depth = configs.e_layers

        for _ in range(self.depth):
            self.PatchMixer_blocks.append(
                PatchMixerLayer(dim=self.patch_num, a=self.a, kernel_size=self.kernel_size)
            )

        self.W_P = nn.Linear(self.patch_size, self.d_model)
        self.flatten = nn.Flatten(start_dim=-2)

        self.dropout = nn.Dropout(self.dropout_rate)
        self.revin = revin
        if self.revin:
            self.revin_layer = RevIN(self.n_vals, affine=affine, subtract_last=subtract_last)

    def forward(self, x):
        bs = x.shape[0]
        n_vars = x.shape[-1]

        if self.revin:
            x = self.revin_layer(x, 'norm')

        x = x.permute(0, 2, 1)
        x_lookback = self.padding_patch_layer(x)
        x = x_lookback.unfold(dimension=-1, size=self.patch_size, step=self.stride)

        x = self.W_P(x)
        x = torch.reshape(x, (x.shape[0] * x.shape[1], x.shape[2], x.shape[3]))
        x = self.dropout(x)

        for PatchMixer_block in self.PatchMixer_blocks:
            x = PatchMixer_block(x)

        # Global representation (flatten for merge)
        x = self.flatten(x)  # shape: (batch*n_vars, a*d_model)
        x = x.view(bs, n_vars, -1)
        x = x.mean(dim=1)  # aggregate across variables
        return x  # shape: (batch, patch_num*d_model)


# Full Model: PatchMixer + Feature Branch
class PatchMixerFeatureModel(nn.Module):
    def __init__(self, configs, feature_dim=4):
        super().__init__()
        self.backbone = PatchMixerBackbone(configs)

        # Feature Branch
        self.feature_mlp = nn.Sequential(
            nn.Linear(feature_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU()
        )

        # Combined Head
        patch_repr_dim = self.backbone.a * self.backbone.d_model
        self.fc = nn.Sequential(
            nn.Linear(patch_repr_dim + 8, 64),
            nn.ReLU(),
            nn.Linear(64, configs.pred_len)  # output = horizon length
        )

    def forward(self, ts_input, feature_input):
        patch_repr = self.backbone(ts_input)
        feature_repr = self.feature_mlp(feature_input)
        combined = torch.cat([patch_repr, feature_repr], dim=1)
        out = self.fc(combined)
        return out

In [ ]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.version.cuda)
print(torch.__version__)
print(torch.cuda.get_device_name(0))
print(torch.__version__)

In [ ]:

class Configs:
    enc_in = 1
    seq_len = lookback
    pred_len = horizon
    patch_len = 16
    stride = 8
    mixer_kernel_size = 8
    d_model = 32
    head_dropout = 0.1
    e_layers = 3

configs = Configs()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = PatchMixerFeatureModel(configs).to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

for epoch in tqdm(range(5)):
    print('epoch', epoch)
    model.train()
    total_loss = 0
    for X_ts_batch, X_feat_batch, y_batch in train_loader:
        X_ts_batch, X_feat_batch, y_batch = X_ts_batch.to(device), X_feat_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        preds = model(X_ts_batch, X_feat_batch)
        loss = criterion(preds, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch + 1}, Train Loss: {total_loss / len(train_loader): .4f}")


In [ ]:
model.eval()
predictions = []
with torch.no_grad():
    for X_ts_batch, X_feat_batch, _ in val_loader:
        X_ts_batch = X_ts_batch.to(device)
        X_feat_batch = X_feat_batch.to(device)
        preds = model(X_ts_batch, X_feat_batch)
        predictions.append(preds.cpu().numpy())

predictions = np.concatenate(predictions, axis = 0)
predictions_inverse = scaler_y.inverse_transform(
    predictions.reshape(-1, 1)
).reshape(predictions.shape)

weeks = [f'week_{i+1}' for i in range(horizon)]
result_df = pl.DataFrame({weeks[i]: predictions_inverse[:, i] for i in range(horizon)})
print('예측 결과 샘플')
print(result_df)

In [ ]:
result_df[1]

In [ ]:
result_df[2]

In [ ]:
import matplotlib.pyplot as plt
import random
sample_idx = random.randint(0, len(val_dataset) - 1)

X_ts_sample, X_feat_sample, y_true_sample = val_dataset[sample_idx]
X_ts_sample = X_ts_sample.numpy().squeeze() # (Lookback, )
y_true_sample = y_true_sample.numpy() # (horizon, )

model.eval()
with torch.no_grad():
    X_ts_tensor = torch.tensor(X_ts_sample, dtype=torch.float32).unsqueeze(0).unsqueeze(-1).to(device)
    X_feat_tensor = torch.tensor(X_feat_sample, dtype=torch.float32).unsqueeze(0).to(device)
    y_pred_scaled = model(X_ts_tensor, X_feat_tensor).cpu().numpy().flatten()

y_true = scaler_y.inverse_transform(y_true_sample.reshape(-1, 1)).flatten()
y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()

lookback_series = scaler_x.inverse_transform(X_ts_sample.reshape(-1, 1)).flatten()


plt.figure(figsize = (12, 5))
plt.plot(range(len(lookback_series)), lookback_series, label = 'History (Lookback)', color = 'blue')
plt.plot(range(len(lookback_series), len(lookback_series) + len(y_true)), y_true, label = 'Actual (Horizon)', color = 'green')
plt.plot(range(len(lookback_series), len(lookback_series) + len(y_pred)), y_pred, label = 'Predicted (Horizon)', color = 'red', linestyle = 'dashed')

plt.axvline(len(lookback_series)-1, color = 'gray', linestyle = '--', alpha = 0.5)
plt.title(f"Sample #{sample_idx} - Actual vs Predicted (Horizon = {horizon})")
plt.xlabel('Time Steps (Weeks)')
plt.ylabel('Demand Quantity')
plt.legend()
plt.grid(True)
plt.show()